In [1]:
from tqdm.notebook import tqdm as tqdm
import random
import os
import time
import gc
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import multiprocessing as mp
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
import glob
from copy import deepcopy
from typing import Dict
import pdb
import json
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
dataset_path = "AIR_QUALITY/"
csv_files = glob.glob(f"{dataset_path}*.csv")
csv_files

['AIR_QUALITY/PRSA_Data_Dingling_20130301-20170228.csv',
 'AIR_QUALITY/PRSA_Data_Tiantan_20130301-20170228.csv',
 'AIR_QUALITY/PRSA_Data_Aotizhongxin_20130301-20170228.csv',
 'AIR_QUALITY/PRSA_Data_Guanyuan_20130301-20170228.csv',
 'AIR_QUALITY/PRSA_Data_Changping_20130301-20170228.csv',
 'AIR_QUALITY/PRSA_Data_Wanliu_20130301-20170228.csv',
 'AIR_QUALITY/PRSA_Data_Gucheng_20130301-20170228.csv',
 'AIR_QUALITY/PRSA_Data_Huairou_20130301-20170228.csv',
 'AIR_QUALITY/PRSA_Data_Shunyi_20130301-20170228.csv',
 'AIR_QUALITY/PRSA_Data_Nongzhanguan_20130301-20170228.csv',
 'AIR_QUALITY/PRSA_Data_Wanshouxigong_20130301-20170228.csv',
 'AIR_QUALITY/PRSA_Data_Dongsi_20130301-20170228.csv']

year: year of data in this row  , month: month of data in this row 

day: day of data in this row , hour: hour of data in this row 

PM2.5: PM2.5 concentration (ug/m^3) , PM10: PM10 concentration (ug/m^3)

SO2: SO2 concentration (ug/m^3), NO2: NO2 concentration (ug/m^3)

CO: CO concentration (ug/m^3),  O3: O3 concentration (ug/m^3)

TEMP: temperature (degree Celsius) ,  PRES: pressure (hPa)

DEWP: dew point temperature (degree Celsius),  RAIN: precipitation (mm)

wd: wind direction,  WSPM: wind speed (m/s), station: name of the air-quality monitoring site

In [3]:
cols_all= ['No','year','month','day','hour','PM2.5', 'PM10','SO2','NO2','CO','O3','TEMP','PRES',
           'DEWP','RAIN','wd','WSPM','station']
keep_cols= ['PM2.5','PM10','SO2','NO2','CO','O3', 'TEMP','PRES','DEWP','RAIN','WSPM']

In [4]:
df_concat = pd.concat([pd.read_csv(f)[keep_cols] for f in glob.glob(f"{dataset_path}*.csv")], 
                      keys = cols_all*len(csv_files),ignore_index = True, axis=1,).set_axis(labels=keep_cols*len(csv_files),axis=1)

df_concat.shape

(35064, 132)

In [5]:
df_concat.head()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,...,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
0,4.0,4.0,3.0,NaN,200.0,82.0,-2.3,1020.8,-19.7,0.0,...,9.0,3.0,17.0,300.0,89.0,-0.5,1024.5,-21.4,0.0,5.7
1,7.0,7.0,3.0,NaN,200.0,80.0,-2.5,1021.3,-19.0,0.0,...,4.0,3.0,16.0,300.0,88.0,-0.7,1025.1,-22.1,0.0,3.9
2,5.0,5.0,3.0,2.0,200.0,79.0,-3.0,1021.3,-19.9,0.0,...,7.0,NaN,17.0,300.0,60.0,-1.2,1025.3,-24.6,0.0,5.3
3,6.0,6.0,3.0,NaN,200.0,79.0,-3.6,1021.8,-19.1,0.0,...,3.0,5.0,18.0,NaN,NaN,-1.4,1026.2,-25.5,0.0,4.9
4,5.0,5.0,3.0,NaN,200.0,81.0,-3.5,1022.3,-19.4,0.0,...,3.0,7.0,NaN,200.0,84.0,-1.9,1027.1,-24.5,0.0,3.2


In [6]:
df = pd.concat([pd.read_csv(f) for f in glob.glob(f"{dataset_path}*.csv")], ignore_index = True)
df.shape

(420768, 18)

In [7]:
df.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,3.0,NaN,200.0,82.0,-2.3,1020.8,-19.7,0.0,E,0.5,Dingling
1,2,2013,3,1,1,7.0,7.0,3.0,NaN,200.0,80.0,-2.5,1021.3,-19.0,0.0,ENE,0.7,Dingling
2,3,2013,3,1,2,5.0,5.0,3.0,2.0,200.0,79.0,-3.0,1021.3,-19.9,0.0,ENE,0.2,Dingling
3,4,2013,3,1,3,6.0,6.0,3.0,NaN,200.0,79.0,-3.6,1021.8,-19.1,0.0,NNE,1.0,Dingling
4,5,2013,3,1,4,5.0,5.0,3.0,NaN,200.0,81.0,-3.5,1022.3,-19.4,0.0,N,2.1,Dingling


In [8]:
df['dateInt']=df['year'].astype(str) + df['month'].astype(str).str.zfill(2)+ df['day'].astype(str).str.zfill(2)+df['hour'].astype(str).str.zfill(2)
df['Date'] = pd.to_datetime(df['dateInt'], format='%Y%m%d%H')
df['dateInt']=df['year'].astype(str) + df['month'].astype(str).str.zfill(2)+ df['day'].astype(str).str.zfill(2)
df['date'] = pd.to_datetime(df['dateInt'], format='%Y%m%d')

In [9]:
df.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,...,TEMP,PRES,DEWP,RAIN,wd,WSPM,station,dateInt,Date,date
0,1,2013,3,1,0,4.0,4.0,3.0,NaN,200.0,...,-2.3,1020.8,-19.7,0.0,E,0.5,Dingling,20130301,2013-03-01 00:00:00,2013-03-01
1,2,2013,3,1,1,7.0,7.0,3.0,NaN,200.0,...,-2.5,1021.3,-19.0,0.0,ENE,0.7,Dingling,20130301,2013-03-01 01:00:00,2013-03-01
2,3,2013,3,1,2,5.0,5.0,3.0,2.0,200.0,...,-3.0,1021.3,-19.9,0.0,ENE,0.2,Dingling,20130301,2013-03-01 02:00:00,2013-03-01
3,4,2013,3,1,3,6.0,6.0,3.0,NaN,200.0,...,-3.6,1021.8,-19.1,0.0,NNE,1.0,Dingling,20130301,2013-03-01 03:00:00,2013-03-01
4,5,2013,3,1,4,5.0,5.0,3.0,NaN,200.0,...,-3.5,1022.3,-19.4,0.0,N,2.1,Dingling,20130301,2013-03-01 04:00:00,2013-03-01


In [10]:
# Assuming df is your dataframe and datetime_column is the column containing datetime values
df['week_day'] = df['Date'].dt.day_name()
# If you want the day as an integer (Monday = 0, Sunday = 6)
df['week_day_numeric'] = df['Date'].dt.dayofweek

In [11]:
df.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,...,DEWP,RAIN,wd,WSPM,station,dateInt,Date,date,week_day,week_day_numeric
0,1,2013,3,1,0,4.0,4.0,3.0,NaN,200.0,...,-19.7,0.0,E,0.5,Dingling,20130301,2013-03-01 00:00:00,2013-03-01,Friday,4
1,2,2013,3,1,1,7.0,7.0,3.0,NaN,200.0,...,-19.0,0.0,ENE,0.7,Dingling,20130301,2013-03-01 01:00:00,2013-03-01,Friday,4
2,3,2013,3,1,2,5.0,5.0,3.0,2.0,200.0,...,-19.9,0.0,ENE,0.2,Dingling,20130301,2013-03-01 02:00:00,2013-03-01,Friday,4
3,4,2013,3,1,3,6.0,6.0,3.0,NaN,200.0,...,-19.1,0.0,NNE,1.0,Dingling,20130301,2013-03-01 03:00:00,2013-03-01,Friday,4
4,5,2013,3,1,4,5.0,5.0,3.0,NaN,200.0,...,-19.4,0.0,N,2.1,Dingling,20130301,2013-03-01 04:00:00,2013-03-01,Friday,4


In [12]:
df.set_index('Date', inplace=True)
df.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,...,PRES,DEWP,RAIN,wd,WSPM,station,dateInt,date,week_day,week_day_numeric
Date,,,,,,,,,,,,,,,,,,,,,
2013-03-01 00:00:00,1,2013,3,1,0,4.0,4.0,3.0,NaN,200.0,...,1020.8,-19.7,0.0,E,0.5,Dingling,20130301,2013-03-01,Friday,4
2013-03-01 01:00:00,2,2013,3,1,1,7.0,7.0,3.0,NaN,200.0,...,1021.3,-19.0,0.0,ENE,0.7,Dingling,20130301,2013-03-01,Friday,4
2013-03-01 02:00:00,3,2013,3,1,2,5.0,5.0,3.0,2.0,200.0,...,1021.3,-19.9,0.0,ENE,0.2,Dingling,20130301,2013-03-01,Friday,4
2013-03-01 03:00:00,4,2013,3,1,3,6.0,6.0,3.0,NaN,200.0,...,1021.8,-19.1,0.0,NNE,1.0,Dingling,20130301,2013-03-01,Friday,4
2013-03-01 04:00:00,5,2013,3,1,4,5.0,5.0,3.0,NaN,200.0,...,1022.3,-19.4,0.0,N,2.1,Dingling,20130301,2013-03-01,Friday,4


In [13]:
filtered_df =  df[(df['date'] >= '2013-03-01') & (df['date'] <= '2014-01-01')]

In [14]:
filtered_df.shape, df.shape

((88416, 22), (420768, 22))

In [15]:
filtered_df.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,...,PRES,DEWP,RAIN,wd,WSPM,station,dateInt,date,week_day,week_day_numeric
Date,,,,,,,,,,,,,,,,,,,,,
2013-03-01 00:00:00,1,2013,3,1,0,4.0,4.0,3.0,NaN,200.0,...,1020.8,-19.7,0.0,E,0.5,Dingling,20130301,2013-03-01,Friday,4
2013-03-01 01:00:00,2,2013,3,1,1,7.0,7.0,3.0,NaN,200.0,...,1021.3,-19.0,0.0,ENE,0.7,Dingling,20130301,2013-03-01,Friday,4
2013-03-01 02:00:00,3,2013,3,1,2,5.0,5.0,3.0,2.0,200.0,...,1021.3,-19.9,0.0,ENE,0.2,Dingling,20130301,2013-03-01,Friday,4
2013-03-01 03:00:00,4,2013,3,1,3,6.0,6.0,3.0,NaN,200.0,...,1021.8,-19.1,0.0,NNE,1.0,Dingling,20130301,2013-03-01,Friday,4
2013-03-01 04:00:00,5,2013,3,1,4,5.0,5.0,3.0,NaN,200.0,...,1022.3,-19.4,0.0,N,2.1,Dingling,20130301,2013-03-01,Friday,4


In [16]:
#removing unnecesary columns
cols_to_drop = ["No", 'year', "dateInt", "date"]
filtered_df = filtered_df.drop(cols_to_drop, axis = 1)

In [17]:
filtered_df.head()

,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station,week_day,week_day_numeric
Date,,,,,,,,,,,,,,,,,,
2013-03-01 00:00:00,3,1,0,4.0,4.0,3.0,NaN,200.0,82.0,-2.3,1020.8,-19.7,0.0,E,0.5,Dingling,Friday,4
2013-03-01 01:00:00,3,1,1,7.0,7.0,3.0,NaN,200.0,80.0,-2.5,1021.3,-19.0,0.0,ENE,0.7,Dingling,Friday,4
2013-03-01 02:00:00,3,1,2,5.0,5.0,3.0,2.0,200.0,79.0,-3.0,1021.3,-19.9,0.0,ENE,0.2,Dingling,Friday,4
2013-03-01 03:00:00,3,1,3,6.0,6.0,3.0,NaN,200.0,79.0,-3.6,1021.8,-19.1,0.0,NNE,1.0,Dingling,Friday,4
2013-03-01 04:00:00,3,1,4,5.0,5.0,3.0,NaN,200.0,81.0,-3.5,1022.3,-19.4,0.0,N,2.1,Dingling,Friday,4


In [18]:
# Define the wind_direction_to_trigonometric function
def wind_direction_to_trigonometric(wind_direction):
    angles = {
        'WNW': 292.5, 'NW': 315, 'NE': 45, 'NNE': 22.5,
        'ENE': 67.5, 'N': 0, 'ESE': 112.5, 'NNW': 337.5,
        'E': 90, 'WSW': 247.5, 'W': 270, 'SW': 225,
        'SSW': 202.5, 'SE': 135, 'S': 180, 'SSE': 157.5,
        'NbE': 11.25, 'NEbN': 33.75, 'NEbE': 56.25, 'EbN': 78.75,
        'EbS': 101.25, 'SEbE': 123.75, 'SEbS': 146.25, 'EbN': 78.75,
    }
    
    if wind_direction in angles:
        degrees = angles[wind_direction]
        radians = np.radians(degrees)
        return np.cos(radians), np.sin(radians)
    else:
        return np.nan

In [19]:
filtered_df[['cos_wind_direction', 'sin_wind_direction']] = filtered_df['wd'].apply(wind_direction_to_trigonometric).apply(pd.Series)

In [20]:
TARGET ="PM2.5"
keep_cols = ['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP',
            'RAIN', 'WSPM','cos_wind_direction', 'sin_wind_direction',
            'month', "hour", 'week_day_numeric',TARGET]
cols = ['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP',
        'RAIN', 'WSPM','cos_wind_direction', 'sin_wind_direction',
        'month', "hour", 'week_day_numeric']

In [21]:
air_quality_data = filtered_df[keep_cols]
air_quality_data.shape

(88416, 16)

In [22]:
air_quality_data.head()

,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM,cos_wind_direction,sin_wind_direction,month,hour,week_day_numeric,PM2.5
Date,,,,,,,,,,,,,,,,
2013-03-01 00:00:00,4.0,3.0,NaN,200.0,82.0,-2.3,1020.8,-19.7,0.0,0.5,6.123234e-17,1.000000,3,0,4,4.0
2013-03-01 01:00:00,7.0,3.0,NaN,200.0,80.0,-2.5,1021.3,-19.0,0.0,0.7,3.826834e-01,0.923880,3,1,4,7.0
2013-03-01 02:00:00,5.0,3.0,2.0,200.0,79.0,-3.0,1021.3,-19.9,0.0,0.2,3.826834e-01,0.923880,3,2,4,5.0
2013-03-01 03:00:00,6.0,3.0,NaN,200.0,79.0,-3.6,1021.8,-19.1,0.0,1.0,9.238795e-01,0.382683,3,3,4,6.0
2013-03-01 04:00:00,5.0,3.0,NaN,200.0,81.0,-3.5,1022.3,-19.4,0.0,2.1,1.000000e+00,0.000000,3,4,4,5.0


In [23]:
air_quality_data.values

array([[  4.,   3.,  nan, ...,   0.,   4.,   4.],
       [  7.,   3.,  nan, ...,   1.,   4.,   7.],
       [  5.,   3.,   2., ...,   2.,   4.,   5.],
       ...,
       [139.,  45.,  99., ...,  21.,   2.,  94.],
       [139.,  45.,  99., ...,  22.,   2.,  94.],
       [130.,  49., 100., ...,  23.,   2.,  95.]])

In [24]:
timesteps = 24
prediction_horizon = 1

In [25]:
X_data_imputed = np.zeros((len(air_quality_data), timesteps, air_quality_data.shape[1]-1))
X_data_no_imputed = np.zeros((len(air_quality_data), timesteps, air_quality_data.shape[1]-1))
y_data_imputed = np.zeros((len(air_quality_data), prediction_horizon, 1))
y_data_no_imputed = np.zeros((len(air_quality_data), prediction_horizon, 1))

In [26]:
X_data_imputed.shape ,X_data_no_imputed.shape ,y_data_imputed.shape ,y_data_no_imputed.shape 

((88416, 24, 15), (88416, 24, 15), (88416, 1, 1), (88416, 1, 1))

In [27]:
for i, name in enumerate(cols):
    for j in range(timesteps):
        X_data_imputed[:, j, i] = air_quality_data[name].shift(timesteps - j - 1).fillna(method="bfill")
        X_data_no_imputed[:, j, i] = air_quality_data[name].shift(timesteps - j - 1)
for j in range(prediction_horizon):
    y_data_imputed[:, j, 0] = air_quality_data[TARGET].shift(prediction_horizon - j - 1).fillna(method="bfill")
    y_data_no_imputed[:, j, 0] = air_quality_data[TARGET].shift(prediction_horizon - j - 1)

In [28]:
X_data_imputed.shape ,X_data_no_imputed.shape ,y_data_imputed.shape ,y_data_no_imputed.shape 

((88416, 24, 15), (88416, 24, 15), (88416, 1, 1), (88416, 1, 1))

In [29]:
X_data_imputed = X_data_imputed[timesteps:-prediction_horizon]
X_data_no_imputed = X_data_no_imputed[timesteps:-prediction_horizon]
y_data_imputed = y_data_imputed[timesteps:-prediction_horizon]
y_data_no_imputed = y_data_no_imputed[timesteps:-prediction_horizon]

In [30]:
X_data_imputed.shape ,X_data_no_imputed.shape ,y_data_imputed.shape,y_data_no_imputed.shape

((88391, 24, 15), (88391, 24, 15), (88391, 1, 1), (88391, 1, 1))

In [31]:
y_data_imputed[np.isnan(y_data_imputed)],  y_data_imputed[np.isinf(y_data_imputed)]

(array([], dtype=float64), array([], dtype=float64))

In [32]:
y_data_no_imputed[np.isnan(y_data_no_imputed)],  y_data_no_imputed[np.isinf(y_data_no_imputed)]

(array([nan, nan, nan, ..., nan, nan, nan]), array([], dtype=float64))

In [33]:
X_data_no_imputed[np.isnan(X_data_no_imputed)],  X_data_no_imputed[np.isinf(X_data_no_imputed)]

(array([nan, nan, nan, ..., nan, nan, nan]), array([], dtype=float64))

# Extract the last observation of each feature

In [34]:
nb_pats, seq, n_features = X_data_no_imputed.shape
timeseries_last_obs_data = []
for i in range(nb_pats):
    Index_Last=(~np.isnan(X_data_no_imputed[i,:,:])).cumsum(0).argmax(0)
    Last_Indices = np.reshape(Index_Last,(1,n_features))
    Last_Values = np.take_along_axis(X_data_no_imputed[i,:,:], Last_Indices, axis = 0)
    timeseries_last_obs_data.append(np.repeat(Last_Values, seq, axis=0))
last_obs_data=np.stack(timeseries_last_obs_data)

In [35]:
last_obs_data[np.isnan(last_obs_data)],  last_obs_data[np.isinf(last_obs_data)]

(array([nan, nan, nan, ..., nan, nan, nan]), array([], dtype=float64))

In [36]:
col_median = np.nanmedian(last_obs_data, axis=0)
last_obs_data_imputed = np.where(np.isnan(last_obs_data), 
                         col_median, last_obs_data)

In [37]:
last_obs_data[np.isnan(last_obs_data)],  last_obs_data[np.isinf(last_obs_data)]

(array([nan, nan, nan, ..., nan, nan, nan]), array([], dtype=float64))

In [38]:
last_obs_data_imputed[np.isnan(last_obs_data_imputed)],  last_obs_data_imputed[np.isinf(last_obs_data_imputed)]

(array([], dtype=float64), array([], dtype=float64))

# The frequency observation of each feature 


In [41]:
# isnan
freq_list= []
nb_pats, seq, n_features = X_data_no_imputed.shape
for i in range(nb_pats):
    data_samples=  np.expand_dims(X_data_no_imputed[i,:,:], axis=0)
    nan_counts = np.sum(np.isnan(data_samples), axis=(0, 1))
    freq_list.append(np.repeat(np.expand_dims(nan_counts, axis=0), seq, axis=0))
freqs = np.stack(freq_list)

In [42]:
import math
import re
import itertools  
import datetime
from collections import namedtuple, defaultdict
from tqdm import tqdm
from itertools import groupby
from operator import itemgetter
from functools import partial
from itertools import combinations

In [43]:
def forward_imputation(data):
    store_index = []
    for index in  range(len(data)):
        try: 
            if len(data)>1:
                store_index.append([data[index], data[index+1]])
            else:
                store_index.append(data[index])
        except:
            pass
    return store_index
def _forward_with_last_measured(data, hours_data=48):
    cols_data = data.columns.to_list()
    times_hours_data = np.arange(0,hours_data,1)
    notna_cols_indexes, nan_cols_indexes  = defaultdict(list),defaultdict(list)
    indexes_last_indicator = defaultdict(list)
    for col in cols_data:
        notna_cols_indexes[col].append(list(data.loc[pd.notna(data[col]), :].index))
        nan_cols_indexes[col].append(list(data.loc[pd.isna(data[col]), :].index))
        indexes_last_indicator[col].append([data[col].notna()[::-1].idxmax(),
                                      data[col].isna()[::-1].idxmax()])
    notna_cols_indexes_ = {key: list(itertools.chain.from_iterable(value)) 
                                 for key , value in notna_cols_indexes.items() if list(itertools.chain.from_iterable(value))}
    notna_cols_indexes_ ={ key:value  for key , value in notna_cols_indexes_.items() if len(value)!=len(times_hours_data)}

    nan_cols_indexes = {key: list(itertools.chain.from_iterable(value)) 
                                for key , value in nan_cols_indexes.items()}
    nan_cols_indexes_ ={key:value  for key , value in nan_cols_indexes.items() 
                                if len(value)!=len(times_hours_data)}
    nan_cols_indexes_ ={key:value  for key , value in nan_cols_indexes_.items() if value}
    
    indexes_last_indicator = {key: list(itertools.chain.from_iterable(value)) 
                                    for key , value in indexes_last_indicator.items()}
    matrix_indexes_notna = {key:forward_imputation(value) for key, value in notna_cols_indexes_.items()}
    matrix_notna_with_last = {key:list(itertools.chain.from_iterable([matrix_indexes_notna[key], [indexes_last_indicator[key]]])) 
                                 for key in indexes_last_indicator if key in matrix_indexes_notna }
    final_matrix_indexes = defaultdict(list)
    for key, vals in matrix_notna_with_last.items():
        for val in vals:
            if isinstance(val, list):
                final_matrix_indexes[key].append(val)
    matrix_range_indexes_cols = {key:[final_matrix_indexes[key], nan_cols_indexes_[key]] 
                                 for key in nan_cols_indexes_ if key in final_matrix_indexes}
    
    range_indexes_cols_imputed = {}
    for key, value in matrix_range_indexes_cols.items():
        range_values = [[notna_ind,nan] for nan in value[1] 
                      for notna_ind in value[0] if notna_ind[0]<=nan<=notna_ind[1]]
        range_indexes_cols_imputed[key]=range_values
    return range_indexes_cols_imputed

def forward_with_last_measured_value_with_time_elasped_interval(data_copy, mask_data, hrs_used=48):
    resultats = _forward_with_last_measured(data_copy, hours_data=hrs_used)
    data =data_copy.copy()
    mask_forward =mask_data.copy()
    for key , values in resultats.items():
        for k, group in groupby(values, lambda x:x[0]):
            vals_ = list(itertools.chain.from_iterable(list(group)))
            vals_ = np.array([val for val in vals_ if not isinstance(val, list)])
            if k[1]<hrs_used-1:  
                for index in np.arange(k[0]+1, k[1]):
                    data.at[index, key]= data._get_value(k[0], key)
                    mask_forward.at[index, key]= index-k[0]
            else:
                for index in np.arange(k[0]+1, k[1]+1):
                    if pd.isnull(data._get_value(index, key)):
                        #print(index, key, data[key])
                        data.at[index, key]= data._get_value(k[0], key)
                        mask_forward.at[index, key]= index-k[0]
                    else:
                        pass
    return data, mask_forward  

In [44]:
X_data_no_imputed

array([[[  7.,   3.,  nan, ...,   3.,   1.,   4.],
        [  5.,   3.,   2., ...,   3.,   2.,   4.],
        [  6.,   3.,  nan, ...,   3.,   3.,   4.],
        ...,
        [  5.,   3.,  nan, ...,   3.,  22.,   4.],
        [  9.,   3.,   2., ...,   3.,  23.,   4.],
        [  7.,   3.,   2., ...,   3.,   0.,   5.]],

       [[  5.,   3.,   2., ...,   3.,   2.,   4.],
        [  6.,   3.,  nan, ...,   3.,   3.,   4.],
        [  5.,   3.,  nan, ...,   3.,   4.,   4.],
        ...,
        [  9.,   3.,   2., ...,   3.,  23.,   4.],
        [  7.,   3.,   2., ...,   3.,   0.,   5.],
        [  6.,   3.,   2., ...,   3.,   1.,   5.]],

       [[  6.,   3.,  nan, ...,   3.,   3.,   4.],
        [  5.,   3.,  nan, ...,   3.,   4.,   4.],
        [  6.,   3.,   4., ...,   3.,   5.,   4.],
        ...,
        [  7.,   3.,   2., ...,   3.,   0.,   5.],
        [  6.,   3.,   2., ...,   3.,   1.,   5.],
        [  2.,  13.,  39., ...,   3.,   2.,   5.]],

       ...,

       [[ 90.,   9.,  44

In [45]:
x_features = cols
x_features

['PM10',
 'SO2',
 'NO2',
 'CO',
 'O3',
 'TEMP',
 'PRES',
 'DEWP',
 'RAIN',
 'WSPM',
 'cos_wind_direction',
 'sin_wind_direction',
 'month',
 'hour',
 'week_day_numeric']

In [46]:
timedelta, features_inputs= [],[]
for x_data,freq in tqdm(zip(X_data_no_imputed, freqs), total=len(y_data_imputed),
                            desc='Iterating over samples'):
    masked_timeseries = np.where((pd.isnull(x_data)), np.nan, 0)
    features_data = pd.DataFrame(x_data, columns=x_features)
    masked_episode_timeseries = pd.DataFrame(masked_timeseries, columns=x_features)
    data_for_imp,time_elasped_timeseries=forward_with_last_measured_value_with_time_elasped_interval(features_data,
                                                                                                     masked_episode_timeseries,
                                                                                                     hrs_used=timesteps)
    timedelta.append(time_elasped_timeseries.values)
    features_inputs.append(masked_episode_timeseries.values)

Iterating over samples: 100%|██████████| 88391/88391 [06:43<00:00, 219.09it/s]


In [47]:
delta_time_data=np.stack(timedelta)
mask_data=np.stack(features_inputs)

In [48]:
dataset_name =f"BeijingAirQuality_MultiSite_{timesteps}_timesteps".upper()

In [51]:
dn=f"/data/{dataset_name}"
if not os.path.exists(dn):
    os.makedirs(dn)

In [52]:
dataset_name

'BEIJINGAIRQUALITYMULTISITE_FREQUENCY_WEIGHTING_24_TIMESTEPS'

In [53]:
np.savez(os.path.join(dn,f"data_{timesteps}.npz"),
         last_obs=last_obs_data, 
         last_obs_imputed=last_obs_data_imputed, 
         targets_data=y_data_imputed,
         frequencies_data=freqs, 
         timeseries_data=X_data_no_imputed,
         timeseries_data_imputed=X_data_imputed,
         timedelta_data=delta_time_data) 

In [54]:
delta_time_data[np.isnan(delta_time_data)] = 999
delta_time_data[np.isinf(delta_time_data)] = 999

In [55]:
last_obs_data[np.isnan(last_obs_data)],  last_obs_data[np.isinf(last_obs_data)]

(array([nan, nan, nan, ..., nan, nan, nan]), array([], dtype=float64))

In [56]:
timeseries_data_max = np.nanmax(X_data_no_imputed,axis=0)
timeseries_data_min = np.nanmin(X_data_no_imputed, axis=0)

last_observation_data_max = np.nanmax(last_obs_data,axis=0)
last_observation_data_min = np.nanmin(last_obs_data, axis=0)

In [57]:
features_data = (X_data_no_imputed - timeseries_data_min) / (timeseries_data_max - timeseries_data_min)

last_data = (last_obs_data - last_observation_data_min) / (last_observation_data_max - last_observation_data_min)

In [58]:
last_data[np.isnan(last_data)],  last_data[np.isinf(last_data)]

(array([nan, nan, nan, ..., nan, nan, nan]), array([], dtype=float64))

In [59]:
features_data[np.isnan(features_data)],  features_data[np.isinf(features_data)]

(array([nan, nan, nan, ..., nan, nan, nan]), array([], dtype=float64))

In [60]:
TARGETS =np.squeeze(y_data_imputed,axis=-1)
TARGETS

array([[ 4.],
       [ 6.],
       [ 7.],
       ...,
       [66.],
       [94.],
       [94.]])

In [61]:
targets_max = TARGETS.max()
targets_min = TARGETS.min()

targets = (TARGETS - targets_min) / (targets_max - targets_min)

In [69]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset,TensorDataset


def dataloaders_double_cv_cohort(train_x, train_t,  train_last,  train_freq, train_y, 
                          test_x, test_x2, test_t,  test_last,  test_freq,  test_y, BATCH_SIZE=128):
        
    def collate_fn(batch):
        return tuple(zip(*batch))

    train_dataset = TensorDataset(torch.tensor(train_x,dtype=torch.float32),
                                  torch.tensor(train_t,dtype=torch.float32),
                                  torch.tensor(train_last,dtype=torch.float32),
                                  torch.tensor(train_freq,dtype=torch.float32),
                                  torch.tensor(train_y,dtype=torch.float32))  
    
    
    valid_dataset = TensorDataset(torch.tensor(test_x,dtype=torch.float32),
                                  torch.tensor(test_t,dtype=torch.float32),
                                  torch.tensor(test_last,dtype=torch.float32),
                                  torch.tensor(test_freq,dtype=torch.float32),
                                  torch.tensor(test_x2,dtype=torch.float32),
                                  torch.tensor(test_y,dtype=torch.float32))  
    
    
    
    train_data_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=4,
    )

    valid_data_loader = DataLoader(
        valid_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=4,
    )
    
    return train_data_loader, valid_data_loader
def cv_fold_splits_cohorts(data,data_real,time_data,last_data,features_freqs, target, n_fold=2):
    from sklearn.model_selection import KFold
    from sklearn.model_selection import train_test_split
    test_data_loader, training_data, validation_data = [], [], []
    all_test_notes = []
    kfold = KFold(n_splits=n_fold, shuffle=True, random_state=n_fold)
    for index, (train_index, test_index) in enumerate(kfold.split(data, target)):
        print('<-------OUTTER FOLD', index + 1, '------->')
        
        x_train, x_test = data[train_index], data[test_index]
        _, x_test_real = data_real[train_index], data_real[test_index]
        x_train_last, x_test_last = last_data[train_index], last_data[test_index]
        x_train_freq, x_test_freq = features_freqs[train_index], features_freqs[test_index]
        x_train_t, x_test_t = time_data[train_index], time_data[test_index]
       
        y_train, y_test = target[train_index], target[test_index]
        _, test_loader =  dataloaders_double_cv_cohort(x_train, x_train_t,  x_train_last,
                                                x_train_freq,  y_train, 
                                                x_test, x_test_real, x_test_t,   x_test_last, 
                                                x_test_freq,  y_test)
        
        X_train, X_valid, y_train_fold, y_valid_fold = train_test_split(list(zip(x_train,x_train_t,
                                                                                 x_train_last,x_train_freq,
                                                                                 )), y_train, 
                                                                        test_size=0.2, 
                                                                        random_state=n_fold)
        #Training data
        x_train_in =np.array([X_train[i][0] for i in range(len(X_train))])
        x_train_in_t= np.array([X_train[i][1] for i in range(len(X_train))])
        x_train_in_last= np.array([X_train[i][2] for i in range(len(X_train))])
        x_train_in_freq= np.array([X_train[i][3] for i in range(len(X_train))])
        # Validation data
        x_val_in = np.array([X_valid[i][0] for i in range(len(X_valid))])
        x_val_in_t = np.array([X_valid[i][1] for i in range(len(X_valid))])
        x_val_in_last = np.array([X_valid[i][2] for i in range(len(X_valid))])
        x_val_in_freq = np.array([X_valid[i][3] for i in range(len(X_valid))])
        #print(x_val_in_adm_t.shape, x_val_in_adm_t)
        train_loader, val_loader =  dataloaders_double_cv_cohort(x_train_in, x_train_in_t, x_train_in_last, 
                                                          x_train_in_freq,  y_train_fold ,
                                                          
                                                          x_val_in,x_val_in, x_val_in_t,x_val_in_last,
                                                          x_val_in_freq, y_valid_fold)
        
        training_data.append(train_loader)
        validation_data.append(val_loader)
        
        test_data_loader.append(test_loader)
    return test_data_loader, training_data, validation_data, x_train_in.shape 

In [70]:
targets

array([[0.0023753 ],
       [0.00475059],
       [0.00593824],
       ...,
       [0.0760095 ],
       [0.10926366],
       [0.10926366]])

In [71]:
test_loader, train_loader, valid_loader,data_shape = cv_fold_splits_cohorts(features_data,
                                                                            X_data_no_imputed,   
                                                                            delta_time_data,
                                                                            last_data, 
                                                                            freqs,targets,
                                                                            n_fold=10)

<-------OUTTER FOLD 1 ------->
<-------OUTTER FOLD 2 ------->
<-------OUTTER FOLD 3 ------->
<-------OUTTER FOLD 4 ------->
<-------OUTTER FOLD 5 ------->
<-------OUTTER FOLD 6 ------->
<-------OUTTER FOLD 7 ------->
<-------OUTTER FOLD 8 ------->
<-------OUTTER FOLD 9 ------->
<-------OUTTER FOLD 10 ------->


In [ ]:
seq_length = features_data.shape[1]
input_dim = features_data.shape[-1]
hidden_dim, output_dim  = 64, 1
seq_length, input_dim, hidden_dim, output_dim, data_shape
seq_length, input_dim, hidden_dim, output_dim, data_shape
taskname=f"BeijingAirQuality_{seq_length}_HRS_DATA_128".upper()
task=f"{os.path.join(dn, f'{taskname}')}"
if not os.path.exists(task):
    os.makedirs(task)
task

In [ ]:
features_data[np.isnan(features_data)],  features_data[np.isinf(features_data)]

In [ ]:
train_val_loader = random.sample(list(zip(train_loader, valid_loader)), len(train_loader))
np.savez(os.path.join(task, f"train_test_data.npz"), 
            folds_data_test= test_loader,
            folds_data_train_valid= train_val_loader,)
        
np.savez(os.path.join(task, f"data_max_min.npz"), 
         data_max=timeseries_data_max, data_min=timeseries_data_min,
         data_targets_max=targets_max, data_targets_min=targets_min,
         shape_data=data_shape,seq_length=seq_length,
         input_dim=input_dim)
input_dim